In [ ]:
import re
import pandas as pd
import fitz  # PyMuPDF
import os

def extract_circuit_numbers(pdf_path):
    """
    Extraire tous les numéros de circuit du PDF
    """
    circuit_info = []
    
    # Ouvrir le PDF avec PyMuPDF
    doc = fitz.open(pdf_path)
    
    for page_num in range(len(doc)):
        page = doc[page_num]
        text = page.get_text()
        
        # Rechercher des patterns comme "7/W0007,COFLRYB-0.35,GY/W"
        for match in re.finditer(r'(\d+)/W\d+,COFLRYB', text):
            circuit_num = match.group(1)
            
            # Obtenir les coordonnées du texte pour permettre l'annotation
            # On recherche le texte exact pour obtenir sa position
            text_instances = page.search_for(match.group(0))
            
            if text_instances:
                # Prendre la première instance (s'il y en a plusieurs avec le même texte)
                position = text_instances[0]
                
                circuit_info.append({
                    'page_num': page_num,
                    'circuit_number': circuit_num,
                    'match_text': match.group(0),
                    'rect': position  # Coordonnées sous forme de rectangle
                })
    
    doc.close()
    return circuit_info

def match_with_excel(circuit_info, excel_path, circuit_col='Numéro Circuit', sn_col='SN FILS SIMPLE'):
    """
    Correspondre les numéros de circuit avec le fichier Excel
    """
    # Lire le fichier Excel
    df = pd.read_excel(excel_path)
    
    # Convertir la colonne de numéro de circuit en type numérique si nécessaire
    if df[circuit_col].dtype != 'int64':
        df[circuit_col] = pd.to_numeric(df[circuit_col], errors='coerce')
    
    # Ajouter les informations SN FILS SIMPLE à chaque entrée de circuit
    for entry in circuit_info:
        circuit_num = int(entry['circuit_number'])
        
        # Chercher dans Excel
        matching_row = df[df[circuit_col] == circuit_num]
        
        if not matching_row.empty:
            entry['sn_fils_simple'] = str(matching_row.iloc[0][sn_col])
        else:
            entry['sn_fils_simple'] = "Non trouvé"
    
    return circuit_info

def add_annotations_to_pdf(pdf_path, output_path, annotations):
    """
    Ajouter des annotations au PDF existant
    """
    # Ouvrir le PDF
    doc = fitz.open(pdf_path)
    
    # Grouper les annotations par page
    annotations_by_page = {}
    for ann in annotations:
        page_num = ann['page_num']
        if page_num not in annotations_by_page:
            annotations_by_page[page_num] = []
        annotations_by_page[page_num].append(ann)
    
    # Parcourir chaque page et ajouter les annotations
    for page_num in range(len(doc)):
        if page_num in annotations_by_page:
            page = doc[page_num]
            
            for ann in annotations_by_page[page_num]:
                circuit_num = ann['circuit_number']
                sn_fils = ann['sn_fils_simple']
                rect = ann['rect']  # Coordonnées du texte trouvé
                
                # Calculer la position de l'annotation
                # L'annotation sera placée à droite du texte trouvé
                x0, y0, x1, y1 = rect
                
                # Texte à ajouter
                annotation_text = f"\t\t\t{sn_fils}"
                
                # Ajouter l'annotation (texte en rouge)
                text_point = fitz.Point(x1 + 5, y0 + (y1 - y0)/2)  # Décalage à droite du texte original
                page.insert_text(
                    text_point,
                    annotation_text,
                    fontsize=8,
                    color=(1, 0, 0)  # Rouge (R,G,B)
                )
    
    # Enregistrer le PDF modifié
    doc.save(output_path)
    doc.close()
    
    return True

def main():
  
    
    pdf_path = r"C:\Users\user\Desktop\App bouhamidi\2D0070266P-F0000_SpliceDiagram.pdf"
    excel_path = r"C:\Users\user\Desktop\App bouhamidi\LISTE CIRCUIT PLAFONNIER BELIZ.xlsx"
    # Créer un nom pour le fichier de sortie basé sur le nom original
    base_name = os.path.splitext(pdf_path)[0]
    output_path = f"{base_name}_avec_SN_FILS.pdf"
    
    # Colonnes dans le fichier Excel (à adapter à votre fichier)
    circuit_column = "Wire Internal Name"
    sn_column = "SN FILS SIMPLE"
    
    print(f"Extraction des numéros de circuit du fichier {pdf_path}...")
    circuit_info = extract_circuit_numbers(pdf_path)
    print(f"{len(circuit_info)} numéros de circuit trouvés.")
    
    print(f"Recherche des correspondances dans {excel_path}...")
    matched_info = match_with_excel(circuit_info, excel_path, circuit_column, sn_column)
    
    print(f"Ajout des annotations au PDF...")
    add_annotations_to_pdf(pdf_path, output_path, matched_info)
    
    print("Traitement terminé avec succès!")
    print(f"Résultat enregistré dans: {output_path}")

if __name__ == "__main__":
    main()

Extraction des numéros de circuit du fichier C:\Users\user\Desktop\App bouhamidi\2D0070266P-F0000_SpliceDiagram.pdf...
79 numéros de circuit trouvés.
Recherche des correspondances dans C:\Users\user\Desktop\App bouhamidi\LISTE CIRCUIT PLAFONNIER BELIZ.xlsx...
Ajout des annotations au PDF...
Traitement terminé avec succès!
Résultat enregistré dans: C:\Users\user\Desktop\App bouhamidi\2D0070266P-F0000_SpliceDiagram_avec_SN_FILS.pdf
